# Finetuning Gemma Model

In [ ]:
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2

!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.0

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.1
    Uninstalling accelerate-0.27.1:
      Successfully uninstalled accelerate-0.27.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.17.0
    Uninstalling datasets-2.17.0:
      Successfully uninstalled datasets-2.17.0


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer


In [ ]:
#import hf_token here

In [ ]:
# set hf_token here


# Prerequsites

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,    #The model weighs are in 32 bit we try to load it in 4bit
    bnb_4bit_quant_type="nf4", #quantization technique nf4(4-bit NormalFloat(NF4))
    bnb_4bit_compute_dtype=torch.bfloat16 #new weights will get updated to f16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token = os.environ['HF_TOKEN'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs= tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto, Portugal.

I have always been interested in art, but I never thought I could make a living out of it.


In [ ]:
print(outputs[0])

tensor([     2,  14782, 235292, 122399,    603,    978, 235269,   1178,   5567,
        235265,    109, 235285,   1144,    476,   2011, 235290, 145008,   9161,
        235269,   7565,    575, 235248, 235274, 235315, 235321, 235308,    575,
           573,   4964,   3413,    576,  35639, 235269,  21539, 235265,    109,
        235285,    791,   2593,   1125,   8202,    575,   3096, 235269,    901,
           590,   2447,   3421,    590,   1538,   1501,    476,   5027,    921,
           576,    665, 235265], device='cuda:0')


In [ ]:
os.environ["WANDB_DISABLED"] = 'false'

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules= ["q_proj", "o_proj", "k_proj","v_proj",
                     "gate_proj","up_proj","down_proj"],
    task_type="CASUAL_LM",
)

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data =data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [ ]:
data['train']['quote'][0]

'“Be yourself; everyone else is already taken.”'

In [ ]:
def formatting_func(example):
  text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [ ]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:463: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
1,1.736300
2,0.934000
3,1.535000
4,2.095800
5,1.596300
6,1.994000
7,2.433700
8,1.808700
9,2.683400
10,1.628500


TrainOutput(global_step=100, training_loss=1.7882964819669724, metrics={'train_runtime': 94.1289, 'train_samples_per_second': 4.249, 'train_steps_per_second': 1.062, 'total_flos': 189744345784320.0, 'train_loss': 1.7882964819669724, 'epoch': 0.1594896331738437})

In [ ]:
text2 = "Quote: Twenty years from now you will be more disappointed by the things that you didn't do than by the ones you did do."

device = "cuda:0"
inputs= tokenizer(text2, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Twenty years from now you will be more disappointed by the things that you didn't do than by the ones you did do. So throw off the bowlines. Sail away from the safe harbor. Catch the trade winds in your sails. Explore. Dream. Discover.

-Mark Twain

I'm not sure if I


In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Prints the GPU name


True
NVIDIA L4
